In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [ ]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-23 14:17:43.510004: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-23 14:17:43.510043: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:

nActions = 4
tableSize = 4
hiddenFeatures = 16

# Higher temporal window confuses the model
# temporalWindow = 5
temporalWindow = 1



env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))

# Result look alike
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha

# Worst result
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()

# Less invalid moves with more limit
replayMemory = SequentialMemory(limit=2000, window_length=temporalWindow)

dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=1000,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=20000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2025-03-23 14:18:02.648091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-23 14:18:02.648136: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-23 14:18:02.648152: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a7476f3fb4ff): /proc/driver/nvidia/version does not exist
2025-03-23 14:18:02.648357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA


Training for 20000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   445/20000: episode: 1, duration: 0.301s, episode steps: 445, steps per second: 1477, episode reward: -16184.000, mean reward: -36.369 [-50.000, 68.000], mean action: 1.939 [0.000, 3.000],  loss: --, mae: --, mean_q: --
   723/20000: episode: 2, duration: 0.170s, episode steps: 278, steps per second: 1634, episode reward: -10238.000, mean reward: -36.827 [-50.000, 32.000], mean action: 1.899 [0.000, 3.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1050/20000: episode: 3, duration: 0.661s, episode steps: 327, steps per second: 495, episode reward: -10768.000, mean reward: -32.930 [-50.000, 68.000], mean action: 1.783 [0.000, 3.000],  loss: 896.437793, mae: 11.182580, mean_q: 1.850282
  1629/20000: episode: 4, duration: 2.245s, episode steps: 579, steps per second: 258, episode reward: -19018.000, mean reward: -32.846 [-50.000, 168.000], mean action: 1.304 [0.000, 3.000],  loss: 483.147461, mae: 18.585255, mean_q: -12.395197
  1790/20000: episode: 5, duration: 0.625s, episode steps: 161, steps per second: 258, episode reward: -4408.000, mean reward: -27.379 [-50.000, 36.000], mean action: 2.342 [0.000, 3.000],  loss: 372.735413, mae: 47.114159, mean_q: -51.118969
  2254/20000: episode: 6, duration: 1.773s, episode steps: 464, steps per second: 262, episode reward: -16390.000, mean reward: -35.323 [-50.000, 128.000], mean action: 1.472 [0.000, 3.000],  loss: 382.070038, mae: 80.823059, mean_q: -95.400055
  2576/20000: episode: 7,

# Simulate Game

In [4]:
# Play the game with predicts

def log2_transform(grid):
    transformed = np.where(grid > 0, np.log2(grid), 0)
    return transformed

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(100)]
metrics = {
    "gameId": [],
    "score": [],
    "higherTile": [],
    "invalidMove (%)": [],
    "steps": [],
    "upMoves": [],
    "downMoves": [],
    "leftMoves": [],
    "rightMoves": [],
    "upSeqAvg": [],
    "downSeqAvg": [],
    "leftSeqAvg": [],
    "rightSeqAvg": [],
}
for i, game in enumerate(games):
    # Metrics
    steps = 0
    invalidMove = 0
    upMoves = 0
    downMoves = 0
    leftMoves = 0
    rightMoves = 0
    upSeqAvg = 0
    downSeqAvg = 0
    leftSeqAvg = 0
    rightSeqAvg = 0

    sequences = {
        "up": [],
        "down": [],
        "left": [],
        "right": []
    }
    lastMove = None

    # print(f"Game {i + 1} initial state:")
    # game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        observation = log2_transform(observation)
        predictedAction = dqn.forward(observation)
        
        # print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        if not game.move(actionMapping[predictedAction]):
            invalidMove += 1
        
        # print(f"Game {i + 1} currrent state:")
        # game.render()

        # input("\nPress any key to continue...")

        steps += 1
        if actionMapping[predictedAction] == "up":
            upMoves += 1
            if lastMove == "up":
                sequences["up"][-1] += 1
            else:
                sequences["up"].append(1)
        elif actionMapping[predictedAction] == "down":
            downMoves += 1
            if lastMove == "down":
                sequences["down"][-1] += 1
            else:
                sequences["down"].append(1)
        elif actionMapping[predictedAction] == "left":
            leftMoves += 1
            if lastMove == "left":
                sequences["left"][-1] += 1
            else:
                sequences["left"].append(1)
        elif actionMapping[predictedAction] == "right":
            rightMoves += 1
            if lastMove == "right":
                sequences["right"][-1] += 1
            else:
                sequences["right"].append(1)
        
        lastMove = actionMapping[predictedAction]

    print(f"Game {i + 1} is over.")

    metrics["gameId"].append(i)
    metrics["score"].append(game.score)
    metrics["higherTile"].append(np.max(game.grid))
    metrics["invalidMove (%)"].append(invalidMove/steps)
    metrics["steps"].append(steps)
    metrics["upMoves"].append(upMoves)
    metrics["downMoves"].append(downMoves)
    metrics["leftMoves"].append(leftMoves)
    metrics["rightMoves"].append(rightMoves)
    metrics["upSeqAvg"].append(np.mean(np.array(sequences["up"])))
    metrics["downSeqAvg"].append(np.mean(np.array(sequences["down"])))
    metrics["leftSeqAvg"].append(np.mean(np.array(sequences["left"])))
    metrics["rightSeqAvg"].append(np.mean(np.array(sequences["right"])))


metrics = pd.DataFrame(metrics).set_index("gameId")
display(metrics)
display(metrics.describe())


/tmp/ipykernel_683/802290428.py:4: RuntimeWarning: divide by zero encountered in log2
  transformed = np.where(grid > 0, np.log2(grid), 0)


Game 1 is over.
Game 2 is over.
Game 3 is over.
Game 4 is over.
Game 5 is over.
Game 6 is over.
Game 7 is over.
Game 8 is over.
Game 9 is over.
Game 10 is over.
Game 11 is over.
Game 12 is over.
Game 13 is over.
Game 14 is over.
Game 15 is over.
Game 16 is over.
Game 17 is over.
Game 18 is over.
Game 19 is over.
Game 20 is over.
Game 21 is over.
Game 22 is over.
Game 23 is over.
Game 24 is over.
Game 25 is over.
Game 26 is over.
Game 27 is over.
Game 28 is over.
Game 29 is over.
Game 30 is over.
Game 31 is over.
Game 32 is over.
Game 33 is over.
Game 34 is over.
Game 35 is over.
Game 36 is over.
Game 37 is over.
Game 38 is over.
Game 39 is over.
Game 40 is over.
Game 41 is over.
Game 42 is over.
Game 43 is over.
Game 44 is over.
Game 45 is over.
Game 46 is over.
Game 47 is over.
Game 48 is over.
Game 49 is over.
Game 50 is over.
Game 51 is over.
Game 52 is over.
Game 53 is over.
Game 54 is over.
Game 55 is over.
Game 56 is over.
Game 57 is over.
Game 58 is over.
Game 59 is over.
Game 6

,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
gameId,,,,,,,,,,,,
0,576,64,0.000000,78,20,23,18,17,1.000000,1.000000,1.000000,1.000000
1,988,128,0.475490,204,34,126,26,18,1.096774,3.315789,1.130435,1.000000
2,620,64,0.198113,106,31,53,13,9,1.068966,1.606061,1.083333,1.000000
3,552,64,0.593583,187,129,40,13,5,4.031250,1.333333,1.083333,1.000000
4,944,128,0.595918,245,24,75,122,24,1.043478,2.777778,4.357143,1.043478
...,...,...,...,...,...,...,...,...,...,...,...,...
95,828,64,0.548387,217,77,102,21,17,2.750000,3.187500,1.050000,1.000000
96,644,64,0.187500,112,36,47,16,13,1.200000,1.468750,1.066667,1.083333
97,608,64,0.528736,174,25,120,16,13,1.041667,4.137931,1.000000,1.083333


,score,higherTile,invalidMove (%),steps,upMoves,downMoves,leftMoves,rightMoves,upSeqAvg,downSeqAvg,leftSeqAvg,rightSeqAvg
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,774.920000,78.080000,0.328155,161.670000,33.540000,78.010000,29.540000,20.580000,1.254238,2.340723,1.409042,1.139608
std,376.748387,42.925277,0.207529,88.269566,23.732695,62.247874,23.652544,13.766582,0.586989,1.308818,0.833240,0.380032
min,160.000000,16.000000,0.000000,44.000000,7.000000,11.000000,6.000000,3.000000,1.000000,1.000000,1.000000,1.000000
25%,570.000000,64.000000,0.152030,100.000000,20.000000,31.750000,16.000000,13.000000,1.000000,1.190616,1.000000,1.000000
50%,712.000000,64.000000,0.346473,139.500000,27.000000,56.000000,21.500000,18.000000,1.043478,2.025641,1.062500,1.000000
75%,993.000000,128.000000,0.488353,202.500000,36.250000,112.250000,32.000000,24.000000,1.111111,3.043735,1.188889,1.083333
max,2228.000000,256.000000,0.720379,494.000000,140.000000,285.000000,160.000000,92.000000,4.117647,6.052632,5.000000,3.680000


# Manual Game

In [ ]:
moves = ["up", "left", "right", "down"]
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


